## Configuração Inicial

Configuração e import de bibliotecas.

In [4]:
import tensorflow as tf
from os import getcwd
from tensorflow import keras
from funcoes_locais import redimensionar_dataset, processar_imagem

diretorio_treino = 'asl-alphabet/asl_alphabet_train/asl_alphabet_train/'
diretorio_teste = 'asl-alphabet/asl_alphabet_test/'
diretorio_atual = getcwd()

## Importação dos dados - via diretório

Essa é a forma ideal de importar os dados, já que ela permite a separação das bases de treino e validação no momento da importação.
Porém, precisamos pegar apenas uma parte das bases de treino e validação para fins de velocidade do treino.
Para isso, será usado o módulo preprocessar_local, irá manter apenas N amostras por classe.

In [5]:
redimensionar_dataset(75)

Dataset redimensionado para 75 amostras por classe.


Agora, será montado o dataset de treino, validação e teste, usando keras.

In [1]:
train_dataset = keras.utils.image_dataset_from_directory(
    diretorio_treino,
    labels='inferred',
    label_mode='int',
    class_names=None,
    color_mode='rgb',
    batch_size=32,
    image_size=(200, 200),
    shuffle=True,
    seed=42,
    validation_split=0.25,
    subset='training',
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False
)


val_dataset = keras.utils.image_dataset_from_directory(
    diretorio_treino,
    labels='inferred',
    label_mode='int',
    class_names=None,
    color_mode='rgb',
    batch_size=32,
    image_size=(200, 200),
    shuffle=True,
    seed=42,
    validation_split=0.25,
    subset='validation',
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False
)


test_dataset = keras.utils.image_dataset_from_directory(
    diretorio_teste,
    labels=list(range(0,28)),
    label_mode='int',
    class_names=None,
    color_mode='rgb',
    batch_size=32,
    image_size=(200, 200),
    shuffle=False,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False
)

NameError: name 'keras' is not defined

Tendo as imagens carregadas, vamos aplicar a função de pré-processamento, para assim podermos alimentar o modelo:

In [7]:
train_ds = train_dataset.map(lambda x, y: tf.py_function(processar_imagem, [x, y], [tf.float32, tf.int32]))
val_ds = val_dataset.map(lambda x, y: (tf.py_function(processar_imagem, [x, y], [tf.float32, tf.uint32])))
test_ds = test_dataset.map(lambda x, y: (tf.py_function(processar_imagem, [x, y], [tf.float32, tf.int32])))

In [8]:
#  Configuração de Pipeline

AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.cache().shuffle(100).prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

num_classes = 29

pretrained_base = tf.keras.applications.inception_v3.InceptionV3(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=(200, 200, 3),
    pooling='max',
    classes=num_classes,
    classifier_activation='softmax'
)

pretrained_base.Trainable = False

model = tf.keras.Sequential([
    pretrained_base,
    tf.keras.layers.Flatten(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(rate=0.3),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

early_stopping = EarlyStopping(
    min_delta=0.00001,
    monitor='val_loss',
    patience=16,
    restore_best_weights=True,
    mode='min'
)

check_point = ModelCheckpoint(
    diretorio_atual,
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='min'
)

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.7,
                              patience=5,
                              min_lr=0.0001)

model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.007),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
epochs = 6

history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epochs,
    callbacks=[early_stopping, check_point, reduce_lr],
    workers=3
)

Epoch 1/6
51/51 [==============================] - ETA: 0s - loss: 3.8881 - accuracy: 0.0551
Epoch 1: val_loss improved from inf to 3.52902, saving model to C:\Users\Kafat\PycharmProjects\ProjetoAvanti\Checkpoint_model.hdf5
51/51 [==============================] - 46s 671ms/step - loss: 3.8881 - accuracy: 0.0551 - val_loss: 3.5290 - val_accuracy: 0.0203 - lr: 0.0070
Epoch 2/6
51/51 [==============================] - ETA: 0s - loss: 4.0100 - accuracy: 0.0447
Epoch 2: val_loss improved from 3.52902 to 3.43768, saving model to C:\Users\Kafat\PycharmProjects\ProjetoAvanti\Checkpoint_model.hdf5
51/51 [==============================] - 30s 599ms/step - loss: 4.0100 - accuracy: 0.0447 - val_loss: 3.4377 - val_accuracy: 0.0460 - lr: 0.0070
Epoch 3/6
51/51 [==============================] - ETA: 0s - loss: 4.1084 - accuracy: 0.0343
Epoch 3: val_loss did not improve from 3.43768
51/51 [==============================] - 29s 571ms/step - loss: 4.1084 - accuracy: 0.0343 - val_loss: 3.4663 - val_acc